<a href="https://colab.research.google.com/github/BrunoLad/tcc-ufabc/blob/main/tcc_ufabc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Monta o google drive para leitura de dados

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Monta CSV com a lista de todos os livros presente na base, separando-o em um par de nome e título.

In [ ]:
import csv
import os

fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/txt')
authorBookList = map(lambda file: file.split('___', 1), fileList)
with open(os.getcwd() + '/books.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Autor', 'Título'])
    for pair in authorBookList:
        writer.writerow(pair)

In [ ]:
import os
import re

def extract_line_from_file(file_name, string_to_search):
  with open(file_name, 'r') as read_obj:
    for line in read_obj:
      if string_to_search in line:
        return line
  return ''

pattern = r'[^_]*$'
regex = re.compile(pattern, re.MULTILINE)
fileList = os.listdir('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/txt')
bookTitles = map(lambda f: re.search(pattern, f).group().rstrip('.txt'), fileList)
bookList = list(bookTitles)
print(len(bookList))
bookdata = []

for book in bookList:
  line = extract_line_from_file('/content/drive/MyDrive/TCC/Gutenberg-q2nlbr/Gutenberg/GUTINDEX.ALL06072021.txt', book)
  match = re.search(r'\d+', line)
  if match:
    bookdata.append(match.group())

print(len(bookdata))
print(bookdata)